In [1]:
# this notebook shall only define functions (and maybe classes)

In [2]:
# make a Python list of the files in the directory
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
import os
def findfiles(directory):
    objects = os.listdir(directory)  # find all objects in a dir

    files = []
    for i in objects:  # check if very object in the folder ...
        if isFile(directory + i):  # ... is a file.
            files.append(i)  # if yes, append it.
    return files

def isFile(object):
    try:
        os.listdir(object)  # tries to get the objects inside of this object
        return False  # if it worked, it's a folder
    except Exception:  # if not, it's a file
        return True

In [3]:
# processing images into a feature vector
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from sklearn.preprocessing import normalize # lol why numpy don't have normalize

# processing images (given image path) into a feature vector
def calc_feature_vector(model, img_path, fromarray = False):
    img = image.load_img(img_path,target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return normalize(model.predict(x))[0]

Using TensorFlow backend.


In [4]:
import numpy
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K

def init_model():
    model = vgg16.VGG16(weights='imagenet', include_top=True)

    model.layers.pop()
    model.layers.pop()

    new_layer = Dense(10, activation='softmax', name='my_dense')

    inp = model.input
    out = new_layer(model.layers[-1].output)

    # model2 = Model(inp, out)
    
    return model

In [5]:
def get_bounding_boxes(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    cp_height, cp_breath = np.shape(gray)
    ret,thresh = cv2.threshold(gray,225,255,cv2.THRESH_BINARY_INV)

    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #     img_ann = img
    output_boxes = []
    
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if w > cp_height/2.5 and h > cp_breath/2.5:
    #       cv2.rectangle(img_ann,(x,y),(x+w,y+h),(0,0,255),3)
            output_boxes.append([x,y,w,h])
    if output_boxes == []:
        output_boxes.append([0,0,cp_breath,cp_height])
    return output_boxes

In [3]:
import subprocess
import csv
from matplotlib.pyplot import imshow
from PIL import Image
import matplotlib.pyplot as plt
import cv2

def analyse_image(model, path_image_to_analyse, img=None, plotting=False):
    '''
    path_image_to_analyse is the local relative path
    '''
    if img == None:
        img = cv2.imread(path_image_to_analyse)
    
    output_boxes = get_bounding_boxes(img)
    
    result = []
    
    if plotting:
        print("============ FRAME ANALYSED =============")
        imshow(img)
        plt.axis("off")
        plt.show()
        print("----------- NEAREST PICTURES ------------")

    for i, box in enumerate(output_boxes):
        [x,y,w,h] = box
        output_img = np.array(img[y:y+h, x:x+w])
        cv2.imwrite("temp.jpg",output_img)
        imgsearch = calc_feature_vector(model, "temp.jpg")

        match = [imgsearch.dot(fv) for fv in feature_vectors]
        top4 = np.argpartition(match,np.arange(-4,0,1))[-4:][::-1]
        print(top4)
        
    for pic in top4:
        print(pic)
        print("percentage match: {}".format(match[pic]))
        result.append([match[pic], pic])
        
        if plotting:
            imshow(np.asarray(Image.open('database/' + df['img_file_name'][pic], 'r')))
            plt.axis("off")
            plt.show()
        
        
    print("\n\n\n\n\n\n\n\n\n\n")
    
    return result